# SIO113: Assignment #9

#### Instructor: Dave May (dmay@ucsd.edu)

#### Assistant: Gabrielle Hobson (ghobson@ucsd.edu)

#### Scripps Institution of Oceanography, UCSD, Spring 2022

----

<div class="alert alert-block alert-info"><b>Preliminary:</b>
Create a new notebook and rename it using the format YourLastname_FirstInitial_HW_09. 
</div>

<div class="alert alert-block alert-warning"><b>Note:</b>
This assignment requires the following packages / modules / methods
    
`
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import dem_utils as dem_utils
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
`    
</div>

### 1. 3D topographic plotting

1. Following the lecture on digital elevation models, load the topography dataset of the Rocky's `../Datasets/topo_co.nc` using NetCDF. The dataset differs from the lecture, but the procedure is the same.
The variable named `'elevation'` read from the NetCDF file will need to "reshaped" using `np.reshape()` as per the lecture.
2. Create a meshgrid object using `lon_1d`, `lat_1d` using `np.meshgrid()`.
3. Using `from mpl_toolkits.mplot3d import Axes3D`, create a 3D interactive plot using *both*
`ax.plot_surface()` and `ax.contour3D()` *in the same plot*. Make the surface transparent via the `alpha` argument and set its color to grey or black. Plot 20 evenly spaced contours from the range $z = [0, 3500]$ m. Color the contours using the color map created by `dem_utils.create_topo_cmap()`. Add a color bar to your plot, label the axis and include the units of all physical quantities.
4. In part 3, it is likely the result is hard to interpret due to the high frequency variations of the topography. For the purpose of visualization, it is sometimes desirable to smooth the data a little - just for the sake of making a nicer picture. One way to do that is using Gaussian blurring. To apply Gaussian blur use the code snippet shown below. This code will process `z_2d` and return a blurred version `z_2d_b`. Re-do part 3 using `z_2d_b` and compare the results.

```python
# Apply Gaussian blur
from scipy.ndimage import gaussian_filter

z_2d_b = gaussian_filter(z_2d, sigma=4)
```

### 2. Hillslope diffusion applied to Sierra Nevada

The Sierra Nevada is actively uplifting by 1-2 mm/yr.
We wish to use following *simplified* landscape evolution model which only includes hillslope erosion
to simulate the topography over time:

$$
\frac{\partial H}{\partial t} = \kappa \nabla^2 H + U,
$$

where $U$ is the uplift velocity (here assumed to be constant throughout the model).
This is the exact same equation as used in the lecture, excepted we have dropped the fluvial erosion term and assumed $\kappa$ is a constant (in space and time).

Whilst we know the uplift $U$ is approximately 1-2 mm/yr, the diffusion constant $\kappa$ is poorly constrained.
It has been suggested that the Sierra Nevada formed in less than 3 million years.
In the context of our simple landscape evolution model, we wish to use the uplift and age
to try and determine possible values of $\kappa$ which lead to a topograhic expressions
that are similar to the present day Sierra Nevada.


For each of the subsequent parts of this question, you should use the code block below containing the comment at the top  
```python
# --- [CODE BLOCK] "Python code to define and run the LEM" --- #
```

In [ ]:
# --- [CODE BLOCK] "Python code to define and run the LEM" --- #

# ---- INPUT PARAMETERS TO VARY ---- #
#U = INSERT VALUE AND UNCOMMENT LINE      # Units: (m / yr)
#
#kappa = INSERT VALUE AND UNCOMMENT LINE  # Units: (m^2 / yr)
# ---------------------------------- #

# Stop the simulation when time exceeds max_time_myr.
max_time_myr = 400.0 # (Myr)

# Define the size of the domain.
Lx, Ly = 600.0e3, 300.0e3 # (m)

nx, ny = 40, 20 # Number of cells in x, y.

dx, dy = Lx / float(nx), Ly / float(ny) # Size of each FD cell.

# 1D coordinate arrays for x and y
x_1d, y_1d = np.linspace(0.5 * dx, Lx - 0.5 * dx, nx), np.linspace(0.5 * dy, Ly - 0.5 * dy, ny)

# Define an initial condition for H.
np.random.seed(0)
delta_H = 1.0
H_ic = delta_H * np.random.random((nx, ny))

kappa_2d, source_2d, uplift_2d = np.zeros((nx, ny)), np.zeros((nx, ny)), np.zeros((nx, ny))

# Define the uplift function.
for j in range(ny):
    for i in range(nx):
        eax, eay = 0.5, 0.3 # Parameters for the ellipse size
        norm_x, norm_y = x_1d[i] / Lx, y_1d[j] / Ly # Normalize coordinates
        radius = np.sqrt( ((norm_x - 0.5)/eax)**2 + ((norm_y - 0.5)/eay)**2 )
        uplift_2d[i, j] = U * 0.5 * (np.arctan(-120.0 * (radius - 0.5))/(0.5 * np.pi) + 1.0)

        
# Insert [CODE BLOCK] "Python code to view the initial condition" 
# if want to see what the initial condition looks like.
        
# -------------------------------------
# Time loop
# -------------------------------------
tk, time = 0, 0.0
H_2d = np.copy(H_ic)

for tk in range(1, 1000000):
    H_old = np.copy(H_2d)

    kappa_2d[:,:] = kappa    
    source_2d[:,:] = uplift_2d

    dt, H_new = dem_utils.diffusion_equation_perform_one_step(
                                dx, dy, kappa_2d, H_old,
                                source=source_2d,  # Provide a non-zero source
                                dbc_bottom=True, dbc_top=True, dbc_left=True, dbc_right=True, dt_max=2000)
    
    H_2d = np.copy(H_new)    
    
    time += dt
    time_kyr = time * 1.0e-3
    
    # Report info about simulation progress
    if tk % 200 == 0: 
        print('  computed step', tk, 'time', ('%1.2e'%time_kyr), '(kyr)') 
        
    # Abort time loop as simulation makes no sense
    if time * 1.0e-6 > max_time_myr:
        print('==> Simulation reached max time ' + str(max_time_myr) + ' Myr - stop!')
        break

If you are curious what the intiial condition looks like, you can view it using the code block below

In [ ]:
# --- [CODE BLOCK] "Python code to view the initial condition" --- #

# Create a subplot with 2 figures showing the initial condition for H
# and the imposed uplift.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

im1 = ax1.imshow(H_ic.T, origin='lower', extent=(0, Lx, 0, Ly), cmap=plt.cm.RdYlGn_r)
fig.colorbar(im1, ax=ax1);
ax1.set(xlabel='$x$ (m)', ylabel='$y$ (m)', title='$H$ (m)')

im3 = ax2.imshow(uplift_2d.T, origin='lower', extent=(0, Lx, 0, Ly), cmap='binary')
fig.colorbar(im3, ax=ax2);
ax2.set(xlabel='$x$ (m)', ylabel='$y$ (m)', title='$U$ (m/yr)')
ax2.set(yticks=[])
# ------------------------------------------------------------ #

----

### Part 1. 

Modify the code provided to do the following:
1. Within the time loop, compute the maximum topography. Store the result in a variable called `max_topo`. The topography is stored in the variable called `H_2d` and the units of elevation stored in `H_2d` is meters.
2. Within the `if` statement shown below
```python
# Report info about simulation progress
if tk % 200 == 0: 
    print('  computed step', tk, 'time', ('%1.2e'%time_kyr), '(kyr)') 
```
print out the value of `max_topo` so that you can monitor the maximum topography during the simulation.
3. Store the time (`time_kyr`) and the maximum topography (`max_topo`) at each time step.
4. Abort the time loop (using `break`) if the maximum topography exceeds 10 km.
5. After the time loop has finished, create a plot showing the maximum topography (y-axis) over time (x-axis). Time should be reported in kyr, elevation should be shown in meters. Label all axis, and provide units. Add a title to the plot. Include in the plot title, the value of $\kappa$ and $U$ being used.

### Part 2.

Using an uplift of $U$ = 2 mm/yr, find a value of $\kappa$ which at steady-state, 
produces a maximum topography of approximately 4000 m. 
Report the time required to reach steady-state in millions of years.
Given the suggestion that the Sierra Nevada formed in 3 million years, does our simple model indicate that the landscape is presently in a steady-state?

1. You should consider values of $\kappa$ in the range of $10^{-1}$ m$^2$/yr to $10^5$ m$^2$/yr.
2. A maximum topography in the range of 3800 m - 4200 m will be considered acceptably close. 
3. You can eye-ball the time where the topography reaches steady-state from your plot of max. topography vs time. You many need to adjust the x, y ranges on your plot to estimate the time.

**Comments:**  
- You should copy the code from part 1 and use it again here. 
- Modify the parameters `kappa` and `U` accordingly. The units of `kappa` used in the code are m$^2$/yr and the units of `U` used are m/yr.
- If you notice `max_topo` has stagnated at a value far below 4 km, you should stop the simulation and try a different value of $\kappa$.
- Please provide the plot of max. topo. vs time with your answer. 

### Part 3. 

Using a value of $U$ = 2 mm/yr, determine a value of $\kappa$ which produces approximately 4 km of topography in 3 million years. 
1. Is the $\kappa$ you obtained smaller or larger than the kappa computed in part 2? What is the ratio of the two diffusivities found?
2. On your plot of max. topography vs time, add a new line showing the maximum topography which would occur if $\kappa = 0$, i.e. compute the solution to $\partial H / \partial t = U$ using $H_0(\mathbf x) = 0$. Label the curve obtained from ploting the simulation data as `'LEM'` and the other line as `'zero diffusion'`.
3. The slope of these curves is called the "erosion rate". At 3 Myr, is the "erosion rate" of the simulation (curve labelled `'LEM'`) higher or lower than the "erosion rate" of the `'zero diffusion'` curve? Provide an explanation for the difference between the erosion rate of `'LEM'` compared to `'zero diffusion'`.

**Comments:**
- You should copy the code from part 1 and use it again here. 
- Modify the parameters `kappa` and `U` accordingly.
- You may wish to modify the value of `max_time_myr` to a value slightly larger than 3 million years.

<div class="alert alert-block alert-danger"><b> 
To receive full credit, your notebook:
    
1. Must have the correct name;
2. Must be fully commented;
3. Must run as expected;
4. Must be submitted into Canvas before the deadline.
</b></div>